# AI 생성 이미지 판별 - 데이터 전처리

이 노트북에서는 데이터셋을 Train/Validation/Test로 분할하고, 전처리 파이프라인을 정의합니다.

## 목차
1. 라이브러리 임포트
2. 데이터 경로 설정
3. 데이터 분할 (Train/Val/Test)
4. 분할 결과 확인 및 저장
5. 데이터 분포 시각화
6. Transform 정의 (전처리 파이프라인)

## 1. 라이브러리 임포트

In [ ]:
import os
import random
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# PyTorch imports
import torch
from torchvision import transforms

# 시각화 설정
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# 랜덤 시드 고정 (재현성)
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

print("라이브러리 임포트 완료!")
print(f"Random Seed: {RANDOM_SEED}")

## 2. 데이터 경로 설정

In [ ]:
# 프로젝트 루트 디렉토리
PROJECT_ROOT = Path('.').absolute().parent
DATA_DIR = PROJECT_ROOT / 'data' / 'raw'
OUTPUT_DIR = PROJECT_ROOT / 'data' / 'processed'
RESULTS_DIR = PROJECT_ROOT / 'results' / 'figures'

FAKE_DIR = DATA_DIR / 'FAKE'
REAL_DIR = DATA_DIR / 'REAL'

# 출력 디렉토리 생성
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

print(f"프로젝트 루트: {PROJECT_ROOT}")
print(f"데이터 디렉토리: {DATA_DIR}")
print(f"출력 디렉토리: {OUTPUT_DIR}")
print(f"\nFAKE 디렉토리 존재: {FAKE_DIR.exists()}")
print(f"REAL 디렉토리 존재: {REAL_DIR.exists()}")

## 3. 데이터 분할 (Train/Val/Test)

### 분할 전략
- **Train**: 70% (~28,000장)
- **Validation**: 15% (~6,000장)
- **Test**: 15% (~6,000장)
- **Stratified Split**: 클래스 비율 유지 (FAKE:REAL = 50:50)

In [ ]:
# 이미지 파일 경로 수집
print("이미지 파일 경로 수집 중...")

fake_images = list(FAKE_DIR.glob('*.jpg')) + list(FAKE_DIR.glob('*.png'))
real_images = list(REAL_DIR.glob('*.jpg')) + list(REAL_DIR.glob('*.png'))

print(f"\nFAKE 이미지: {len(fake_images):,}개")
print(f"REAL 이미지: {len(real_images):,}개")
print(f"전체 이미지: {len(fake_images) + len(real_images):,}개")

# 경로와 레이블 생성
image_paths = fake_images + real_images
labels = [0] * len(fake_images) + [1] * len(real_images)  # 0: FAKE, 1: REAL

print(f"\n레이블 분포:")
print(f"  FAKE (0): {labels.count(0):,}개")
print(f"  REAL (1): {labels.count(1):,}개")

In [ ]:
# Train / (Val + Test) 분할 (70% / 30%)
print("데이터 분할 중...\n")

train_paths, temp_paths, train_labels, temp_labels = train_test_split(
    image_paths,
    labels,
    test_size=0.3,
    stratify=labels,
    random_state=RANDOM_SEED
)

print(f"1단계 완료: Train / Temp 분할")
print(f"  Train: {len(train_paths):,}개 ({len(train_paths)/len(image_paths)*100:.1f}%)")
print(f"  Temp:  {len(temp_paths):,}개 ({len(temp_paths)/len(image_paths)*100:.1f}%)")

# Val / Test 분할 (각각 15%)
val_paths, test_paths, val_labels, test_labels = train_test_split(
    temp_paths,
    temp_labels,
    test_size=0.5,  # 30%의 절반 = 15%
    stratify=temp_labels,
    random_state=RANDOM_SEED
)

print(f"\n2단계 완료: Val / Test 분할")
print(f"  Val:   {len(val_paths):,}개 ({len(val_paths)/len(image_paths)*100:.1f}%)")
print(f"  Test:  {len(test_paths):,}개 ({len(test_paths)/len(image_paths)*100:.1f}%)")

## 4. 분할 결과 확인 및 저장

In [ ]:
# 분할 결과 상세 확인
print("=" * 60)
print("데이터 분할 결과 (클래스별)")
print("=" * 60)

def count_labels(label_list):
    fake_count = label_list.count(0)
    real_count = label_list.count(1)
    return fake_count, real_count

train_fake, train_real = count_labels(train_labels)
val_fake, val_real = count_labels(val_labels)
test_fake, test_real = count_labels(test_labels)

print(f"\nTrain Set: {len(train_paths):,}개")
print(f"  FAKE: {train_fake:,}개 ({train_fake/len(train_paths)*100:.1f}%)")
print(f"  REAL: {train_real:,}개 ({train_real/len(train_paths)*100:.1f}%)")

print(f"\nValidation Set: {len(val_paths):,}개")
print(f"  FAKE: {val_fake:,}개 ({val_fake/len(val_paths)*100:.1f}%)")
print(f"  REAL: {val_real:,}개 ({val_real/len(val_paths)*100:.1f}%)")

print(f"\nTest Set: {len(test_paths):,}개")
print(f"  FAKE: {test_fake:,}개 ({test_fake/len(test_paths)*100:.1f}%)")
print(f"  REAL: {test_real:,}개 ({test_real/len(test_paths)*100:.1f}%)")

print("\n" + "=" * 60)
print("✅ 클래스 균형이 모든 세트에서 유지되고 있습니다!")
print("=" * 60)

In [ ]:
# DataFrame 생성
print("\nDataFrame 생성 및 CSV 저장 중...")

train_df = pd.DataFrame({
    'image_path': [str(p) for p in train_paths],
    'label': train_labels
})

val_df = pd.DataFrame({
    'image_path': [str(p) for p in val_paths],
    'label': val_labels
})

test_df = pd.DataFrame({
    'image_path': [str(p) for p in test_paths],
    'label': test_labels
})

# CSV 저장
train_csv = OUTPUT_DIR / 'train.csv'
val_csv = OUTPUT_DIR / 'val.csv'
test_csv = OUTPUT_DIR / 'test.csv'

train_df.to_csv(train_csv, index=False)
val_df.to_csv(val_csv, index=False)
test_df.to_csv(test_csv, index=False)

print(f"\n✅ CSV 파일 저장 완료!")
print(f"저장 위치: {OUTPUT_DIR}")
print(f"  - train.csv ({len(train_df):,}개)")
print(f"  - val.csv ({len(val_df):,}개)")
print(f"  - test.csv ({len(test_df):,}개)")

# 샘플 확인
print("\n=== train.csv 샘플 ===")
print(train_df.head())
print(f"\n레이블 분포:\n{train_df['label'].value_counts()}")

## 5. 데이터 분포 시각화

In [ ]:
# 분할 결과 시각화
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

datasets = ['Train', 'Validation', 'Test']
data = [
    (train_fake, train_real),
    (val_fake, val_real),
    (test_fake, test_real)
]
colors = ['#FF6B6B', '#4ECDC4']

for ax, dataset, (fake, real) in zip(axes, datasets, data):
    bars = ax.bar(['FAKE', 'REAL'], [fake, real], color=colors, alpha=0.7, edgecolor='black', linewidth=2)
    ax.set_title(f'{dataset} Set', fontsize=14, fontweight='bold')
    ax.set_ylabel('이미지 개수', fontsize=12)
    ax.grid(axis='y', alpha=0.3)
    
    # 막대 위에 값 표시
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height):,}',
                ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.suptitle('Train/Validation/Test 분할 결과', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()

# 저장
save_path = RESULTS_DIR / 'data_split_distribution.png'
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f"그래프 저장: {save_path}")

plt.show()

In [ ]:
# 전체 분할 비율 파이 차트
fig, ax = plt.subplots(figsize=(8, 8))

sizes = [len(train_paths), len(val_paths), len(test_paths)]
labels = ['Train\n(70%)', 'Validation\n(15%)', 'Test\n(15%)']
colors = ['#FF9999', '#66B2FF', '#99FF99']
explode = (0.05, 0, 0)

wedges, texts, autotexts = ax.pie(
    sizes,
    labels=labels,
    colors=colors,
    autopct='%1.1f%%',
    startangle=90,
    explode=explode,
    shadow=True,
    textprops={'fontsize': 12, 'fontweight': 'bold'}
)

for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontsize(14)

ax.set_title('데이터셋 분할 비율', fontsize=16, fontweight='bold', pad=20)

# 저장
save_path = RESULTS_DIR / 'data_split_ratio.png'
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f"그래프 저장: {save_path}")

plt.show()

## 6. Transform 정의 (전처리 파이프라인)

### 전처리 단계
1. **리사이징**: 224x224 (ResNet, EfficientNet 등의 표준 입력 크기)
2. **정규화**: ImageNet 평균/표준편차 사용
3. **데이터 증강** (Training만):
   - Random Horizontal Flip
   - Random Rotation
   - Color Jitter

In [ ]:
# ImageNet 평균 및 표준편차
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]
IMAGE_SIZE = 224

# Training Transform (데이터 증강 포함)
train_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

# Validation & Test Transform (증강 없음)
val_test_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

print("Transform 정의 완료!")
print(f"\n이미지 크기: {IMAGE_SIZE}x{IMAGE_SIZE}")
print(f"정규화: ImageNet 평균/표준편차")
print(f"  Mean: {IMAGENET_MEAN}")
print(f"  Std:  {IMAGENET_STD}")

In [ ]:
# Transform 적용 예시 시각화
sample_image_path = train_paths[0]
original_image = Image.open(sample_image_path)

# 원본 이미지 크기
print(f"원본 이미지 크기: {original_image.size}")
print(f"샘플 경로: {sample_image_path.name}")

# Transform 적용 (증강 여러 번)
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
fig.suptitle('데이터 증강 예시 (Training Transform)', fontsize=16, fontweight='bold')

# 원본
axes[0, 0].imshow(original_image)
axes[0, 0].set_title('Original', fontsize=12, fontweight='bold')
axes[0, 0].axis('off')

# 증강된 이미지들
for i, ax in enumerate(axes.flat[1:]):
    # Transform 적용 (ToTensor 및 Normalize 제외)
    augmented = transforms.Compose([
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)
    ])(original_image)
    
    ax.imshow(augmented)
    ax.set_title(f'Augmented {i+1}', fontsize=12)
    ax.axis('off')

plt.tight_layout()

# 저장
save_path = RESULTS_DIR / 'data_augmentation_examples.png'
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f"\n그래프 저장: {save_path}")

plt.show()

## 요약

### ✅ 완료된 작업
1. 데이터 분할 (Train: 70%, Val: 15%, Test: 15%)
2. Stratified split으로 클래스 비율 유지
3. CSV 파일로 저장 (`data/processed/*.csv`)
4. 분할 결과 시각화
5. Transform 정의 (리사이징, 정규화, 데이터 증강)

### 📁 생성된 파일
- `data/processed/train.csv` - Train set 경로 목록
- `data/processed/val.csv` - Validation set 경로 목록
- `data/processed/test.csv` - Test set 경로 목록
- `results/figures/data_split_distribution.png` - 분할 결과 시각화
- `results/figures/data_split_ratio.png` - 분할 비율 파이 차트
- `results/figures/data_augmentation_examples.png` - 데이터 증강 예시

### 🎯 다음 단계
- **Phase 4**: PyTorch Dataset 및 DataLoader 구현
- 이 CSV 파일들을 사용하여 데이터 로딩